In [1]:
# Initial params is from the Registered Meshes

# KNN + Boarder Detection + Normal Direction Constrains

# 1) KNN search: several candidates of corresponding points
# 2) Boarder Detection + Normal Direction constrains => remove some bad correspondences

In [2]:
import math
import pickle
import os
import time
import scipy
import numpy as np
from scipy.optimize import minimize
from sklearn.neighbors import NearestNeighbors

## SMPL model
data_root   = '/home/yan/Data2/NOMO_Project_Summary/'
root_folder = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'
import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model

smpl_model  = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'
scan_folder =  data_root + 'Data/TC2_scan/ori_scan/re_name/dataset01/male/'
out_folder  = root_folder + 'optimized_mesh/dataset01/'

# pre-process initial params Based on Registered-Optimization 
param_folder = out_folder + 'register2Optimized_shape_pose/male_param/'

# load SMPL model
model = load_model(smpl_model)
scan_list = os.listdir(scan_folder)

deg2rad = 1.0 / 180 * math.pi
bnds = ((-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (-90*deg2rad, -30*deg2rad), 
        (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (30*deg2rad,   90*deg2rad),
        (5*deg2rad,   30*deg2rad), (-30*deg2rad,-5*deg2rad), (0*deg2rad,    10*deg2rad),  
        (5*deg2rad,   30*deg2rad), (5*deg2rad,  30*deg2rad), (-10*deg2rad,   0*deg2rad),  
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (0*deg2rad,    15*deg2rad), 
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (-15*deg2rad,   0*deg2rad),
        (0*deg2rad,   10*deg2rad), (0*deg2rad,  10*deg2rad), (-5*deg2rad,    5*deg2rad),
        (-10*deg2rad,  0*deg2rad), (-10*deg2rad, 0*deg2rad), (-10*deg2rad,   0*deg2rad), 
        (-10*deg2rad,  0*deg2rad), (0*deg2rad,  10*deg2rad), (0*deg2rad,    10*deg2rad),
        (-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

In [3]:
def loadTarget(scanFile):
    ''' load NOMO3D scans, return Vertices and Faces'''
    with open(scanFile, 'r') as fp:
        content = fp.readlines()
    content = [x.strip() for x in content]

    targetV = []
    targetF = []
    for line in content:
        CC = line.split()
        if len(CC) > 0:
            if CC[0] == 'v':
                targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])
            elif CC[0] == 'f':
                targetF.append([int(CC[1]), int(CC[2]), int(CC[3])])

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    targetF = np.array(targetF, dtype=int) - 1 # .obj file, is 0-based, started from 0
    
    return targetV, targetF

def loadInitParam(paramFile):
    '''load initial parameters, including shape and pose from the previous experiments'''
    if paramFile == None:
        init_betas = np.zeros(10)
        init_pose = np.zeros(72)
        deg2rad = 1.0 / 180 * math.pi
        init_pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
        init_pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
    else:
        with open(paramFile, 'rb') as f:
            param_data = pickle.load(f)
        param_data = param_data['param_data']
        
        init_pose = param_data[:72]
        init_betas = param_data[72:]
    
    return init_pose, init_betas

def normalize_v3(arr):
    '''Normalize a numpy array of 3 component vectors shape=(n, 3)'''
    lens = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    arr[:, 0] /= lens
    arr[:, 1] /= lens
    arr[:, 2] /= lens
    return arr

def getNormals(V, F):
    ''' Creating a normal per triangle, is done by the cross product of two vectors on this triangle.
    Creating smooth normals, i.e. one normal per vertex is done 
    by averaging the normal of all the triangles one vertex is part of.
    
    https://sites.google.com/site/dlampetest/python/calculating-normals-of-a-triangle-mesh-using-numpy'''
    
    norm = np.zeros(V.shape, dtype=V.dtype)
    # Create an indexed view into the vertex array using the array of three indices for triangles
    tris = V[F] 
    # Calculate the normal for all the triangles, 
    # by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle 
    n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
    n = normalize_v3(n)
    # Multiple triangles would then contribute to every vertex
    norm[F[:, 0]] += n
    norm[F[:, 1]] += n
    norm[F[:, 2]] += n
    norm = normalize_v3(norm)
    
    return norm

def getBoarderV(Faces):
    '''the boarder of the mesh are those edges which belong to only one polygon'''
    boundV = []
    edges01 = np.array([Faces[:, 0], Faces[:, 1]], dtype=int).T
    edges02 = np.array([Faces[:, 0], Faces[:, 2]], dtype=int).T
    edges03 = np.array([Faces[:, 1], Faces[:, 2]], dtype=int).T
    edge = np.sort(np.concatenate((edges01, edges02, edges03), axis=0))
    
    unique_e, n_counts = np.unique(edge, axis=0, return_counts=True)
    boarder_edge = unique_e[n_counts==1, :]
    boundV = np.unique(boarder_edge.flatten())
    
    return boundV

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.absolute(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

In [4]:
def meshDist(param):
    '''the distance between the SMPL model and the target scan'''
    
    model.betas[:] = param[27:]
    model.pose[48:60] = param[:12]
    model.pose[3:18] = param[12:27]
    
    modelF = model.f # 0-based, started from 0
    modelV = model.r - np.mean(model.r, 0)
    modelNorm = getNormals(modelV, modelF)
    
    # KNN correspondence search, find 5 nearest neighbour as the estimated correspondences
    N_neigh = 3
    _, t_idx = kdtree.query(modelV, k=N_neigh)
    
    # the correspondence should : 
    #      1) the similar normal, the angle is small
    #      2) the small distance,
    #      3) not at the boarder,
    
    W = np.ones((6890, 1), dtype=float) # 0 for bad correspondences, otherwise 1
    correspondV = np.zeros((6890, 3), dtype=float)
    
    dist = 0
    for idx in range(6890):
        m_normal = modelNorm[idx, :]
        good_neigh_idx = 0
        minAngle = 100
        for c_id in range(N_neigh):
            neigh_idx = t_idx[idx, c_id]
            t_normal = targetNorm[neigh_idx, :]
            angle = angle_between(t_normal, m_normal)
            if angle <= minAngle:
                minAngle = angle
                correspondV[idx, :] = targetV[neigh_idx, :]
                good_neigh_idx = neigh_idx
                
        dist_v = np.sqrt(np.sum(np.square(modelV[idx, :] - targetV[good_neigh_idx, :])))
        if dist_v > 0.1:           # 10 cm
            W[idx, :] = 0
        if minAngle > math.pi / 4: # 45 degree
            W[idx, :] = 0
        if good_neigh_idx in boarderV:
            W[idx, :] = 0
        
        dist = dist + dist_v * W[idx, :]
    # weighted mean distance between the model and the correspondences vertices
#     dist = np.sum(np.multiply(W, np.sqrt(np.sum(np.square(modelV - correspondV), axis=1)))) / np.count_nonzero(W)
    dist = dist / np.count_nonzero(W)
    
#     print np.count_nonzero(W), dist
    return dist * 1000 # [mm]

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

for scan in scan_list[25:]:
    scan_name = scan[:-4]

    targetV, targetF = loadTarget(scan_folder+scan_name+'.obj') # registered mesh, (6890, 3)
    boarderV = getBoarderV(targetF)                             # the vertices on the boarder edges
    targetNorm = getNormals(targetV, targetF)                   # the normal for every target vertices
    kdtree = scipy.spatial.cKDTree(targetV, leafsize=6890)      # KNN search
    
    paramFile = param_folder + scan_name + '.pkl'               # None for female samples                       
    init_pose, init_betas = loadInitParam(paramFile)            # female does not have the pre-param
    init_param        = np.zeros(37)                            # 27 part pose parameters + 10 shape param
    init_param[:12]   = init_pose[48:60]
    init_param[12:27] = init_pose[3:18]
    init_param[27:]   = init_betas
    
    start_time = time.time()
    res = minimize(meshDist, init_param, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-6, 'disp': True})
    end_time = time.time()
    print scan_name, " : eclipse time [s] : ", end_time - start_time
    
    optimal_param = res.x
    print init_betas
    print optimal_param[27:]
    
    # save
    model.betas[:]    = optimal_param[27:]
    model.pose[48:60] = optimal_param[:12]
    model.pose[3:18]  = optimal_param[12:27]

    with open(out_folder + 'scan2register2optimized_shape_pose/male/' + scan_name + '.obj', 'w') as fp:
        for v in model.r: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

    optimal_param = np.zeros(82)
    optimal_param[72:]   = res.x[27:]
    optimal_param[48:60] = res.x[:12]
    optimal_param[3:18]  = res.x[12:27]

    with open(out_folder + 'scan2register2optimized_shape_pose/male_param/' + scan_name + '.pkl', 'w') as fp:
        pickle.dump({'param_data': optimal_param}, fp)

male_0133  : eclipse time [s] :  4292.57912207
[ 2.38716652  0.479123    0.09196205 -0.35131746  0.27723978 -0.74697773
  0.71334233  0.27535674  0.52397085  1.95924022]
[ 2.48779124  0.33297244  0.06930777 -0.57373811  0.35060994 -0.88564359
  0.78050693  0.407203    0.51045947  1.95250007]
male_0071  : eclipse time [s] :  3894.60883713
[-1.08792129 -0.74564321  0.55580785 -0.50199582  0.04837308 -0.38407259
  0.76955342 -0.15837576 -0.33318831  1.17969956]
[-1.14949942 -0.7773389   0.5518229  -0.5191296   0.04331979 -0.3980272
  0.78068178 -0.14727976 -0.33414412  1.17292393]
male_0044  : eclipse time [s] :  3786.96505094
[ 0.96260226 -1.43415582  0.67403302 -0.21169503  0.39403192  0.40526525
  1.17295865  0.50399262 -0.46832315  1.36377892]
[ 1.18246898 -1.30039587  0.65580776 -0.207147    0.49583485  0.39439113
  1.21556828  0.53402204 -0.50377975  1.34594116]
male_0022  : eclipse time [s] :  3857.5121541
[ 1.51140343e+00 -1.01573632e+00  3.74797411e-01 -1.70194094e-01
  5.6859542

KeyboardInterrupt: 

In [ ]:
print "done"